In [1]:
import ftd2xx  # Thorlabs MFF101
import ftd2xx.defines as constants
import time

In [ ]:
def open_or_closed(motor, vrb=True):
    """determine if motor is open or closed"""
    clsd_stat = b'*\x04\x06\x00\x81P\x01\x00\x02\x00\x00\x90'  # closed byte status
    opn_stat  = b'*\x04\x06\x00\x81P\x01\x00\x01\x00\x00\x90'  # open byte status
    st_bit = b"\x29\x04\x00\x00\x21\x01"  # request status bits
  
    motor.write(st_bit);
    mot_stat = motor.read(12)  # NOTE: EXACTLY 12 bits need to be read. If too many, python will freeze waiting for more. If too few, you won't get them all this time (but will get some next time you read)
    if mot_stat == opn_stat:  # shutter appears to be open
        if vrb:  # verbose
            print('Shutter appears to be open')
            return 1  # 1 for open
        elif mot_stat == clsd_stat:  # shutter appears to be closed
            if vrb:  # verbose
                print('Shutter appears to be closed')
                return 0  # 0 for closed
            else:
                print('I am confused about the shutter position')
                return 2  # 2 for confused

In [2]:
serial = b"37000805"

# Raw byte commands for "MGMSG_MOT_MOVE_JOG".
up_position = b"\x6A\x04\x00\x01\x21\x01"  # x01 up
down_position = b"\x6A\x04\x00\x02\x21\x01"  # x02 down

In [4]:
# Recommended d2xx setup instructions from Thorlabs.
motor = ftd2xx.openEx(serial)
print(motor.getDeviceInfo())
motor.setBaudRate(115200)
motor.setDataCharacteristics(constants.BITS_8, constants.STOP_BITS_1, constants.PARITY_NONE)
time.sleep(.05)
motor.purge()
time.sleep(.05)
motor.resetDevice()
motor.setFlowControl(constants.FLOW_RTS_CTS, 0, 0)
motor.setRts()

# Send raw bytes to USB driver.
motor.write(up_position)
motor.close()

{'type': 5, 'id': 67369712, 'description': b'APT Filter Flipper', 'serial': b'37000805'}
